In [ ]:
import torch
import pandas as pd
from factfinder import (
    EventDetection,
    Geocoder,
    TextClassifier,
    TextClassifierTopics,
)
import geopandas as gpd
import numpy as np

Step 1.
Create a DataFrame with texts

In [2]:
df_texts = pd.read_csv("sample_data.csv")
df_texts.head(5)

,Unnamed: 0,message_id,Дата и время,Текст комментария
0,0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...
1,1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...
2,2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя..."
3,3,3,2023.01.28 12:42,"2) Фурштатская, 17 Здесь прямо-таки умоляю обр..."
4,4,4,2023.01.28 12:45,"3) Фурштатская, 13 Отслоение штукатурного слоя..."


Step 2.
Enter number of categories. Then enter name of the columns with texts and start classification script.

In [3]:
device_type = torch.device("cuda:0")
classification_model = TextClassifier(
    repository_id="Sandrro/text_to_function_v2",
    number_of_categories=1,
    device_type=device_type,
)

df_categorized = df_texts.copy()
df_categorized[["cats", "probs"]] = pd.DataFrame(
    df_categorized["Текст комментария"]
    .progress_map(lambda x: classification_model.run(x))
    .to_list()
)
df_categorized.dropna(subset="cats", inplace=True)
df_categorized.head(3)

100%|██████████| 200/200 [00:03<00:00, 65.45it/s] 


,Unnamed: 0,message_id,Дата и время,Текст комментария,cats,probs
0,0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...,Безопасность,0.433
1,1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...,ЖКХ,0.998
2,2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя...",ЖКХ,0.944


Step 3.
Enter name of the column with texts and start geocoder script

In [4]:
df_geocoded = Geocoder().run(df_categorized, text_column="Текст комментария")
df_geocoded.head(3)

2023-09-05 14:20:30,711 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


100%|██████████| 239/239 [01:09<00:00,  3.43it/s]


,index,Unnamed: 0,message_id,Дата и время,Текст комментария,cats,probs,Street,Numbers,Score,location_options,Location,geometry,level
0,0,0,0,2023.01.26 16:32,Здравствуйте! В Санкт-Петербурге нет Генеральн...,Безопасность,0.433,мойка,,0.997,['набережная реки Мойки Санкт-Петербург Россия'],"набережная реки Мойки, Коломна, округ Коломна,...",POINT (30.27929 59.92712),street
1,1,1,1,2023.01.26 11:55,[club143265175|Центральный район Санкт-Петербу...,ЖКХ,0.998,рубинштейна,2,0.900,['улица Рубинштейна 2 Санкт-Петербург Россия'],"Parivar, 2/45, улица Рубинштейна, Владимирский...",POINT (30.34690 59.93247),house
2,2,2,2,2023.01.28 12:39,"1) Фурштатская, 19 Отслоение штукатурного слоя...",ЖКХ,0.944,фурштатская,,0.784,['Фурштатская улица Санкт-Петербург Россия'],"Фурштатская улица, Литейный округ, Санкт-Петер...",POINT (30.35610 59.94514),street


In [5]:
df_geocoded.drop(columns=["Дата и время"]).explore()

Step 4.
Select GeoDataFrame with categories and coordinates, enter path to the file with population statistics (in our case they are distributed in buildings), name of the city, CRS for this city and minimal count of texts for generation of event. Start the script.

In [ ]:
event_model = EventDetection()
messages, events, connections = event_model.run(
    df_geocoded,
    "F:/Coding/SOIKA/data/raw/population.geojson",
    "Санкт-Петербург",
    32636,
    min_event_size=3,
)

Step 5.
Now you can analyze separate GeoDataFrames with events, connections of these events and processed messages.

In [7]:
events.sort_values(by="risk", ascending=False)

,name,docs,level,id,risk,message_ids,geometry
0,0_снег_мусор_вот_не убирают,45,global,0_global_0,0.285665,"47, 55, 79, 150, 100, 134, 11, 46, 129, 45, 57...",POINT (30.36238 59.93593)
0,0_не_невский_дома_на,120,road,0_road_564.0,0.219186,"32, 91, 120, 176, 116",POINT (30.32840 59.93398)
1,1_рекламой борделей_рекламой_борделей_почему,176,global,1_global_0,0.162990,"149, 176, 137, 196",POINT (30.32840 59.93398)
9,9_неудобства_извинения за предоставленные_изви...,159,global,9_global_0,0.131333,"162, 183, 199, 159, 193, 18",POINT (30.34856 59.91949)
2,2_вас_14_00_лес,77,global,2_global_0,0.123692,"103, 48, 58, 77, 37, 53",POINT (30.31418 59.93388)
...,...,...,...,...,...,...,...
1,1_марата 26_11_марата_марата 26 11,"99, 88, 73",road,1_road_1498.0,0.000063,"48, 99, 88, 73",POINT (30.35288 59.92686)
6,6_все_на_да_да ещё,"125, 134, 43",link,6_link_329.0,0.000055,"125, 134, 43",POINT (30.34856 59.91949)
2,2_143265175_44185_здесь_com wall_com,"181, 177",road,2_road_187.0,0.000050,"181, 177",POINT (29.72413 60.19551)
2,2_то_перед_снег_не,"74, 93, 11",link,2_link_329.0,0.000044,"74, 183, 93, 11",POINT (30.34856 59.91949)


In [8]:
events.explore()

In [9]:
connections

,weight,a,b,geometry
0,8,0_global_0,1_global_0,"LINESTRING (30.362 59.936, 30.328 59.934)"
1,9,0_global_0,2_global_0,"LINESTRING (30.362 59.936, 30.314 59.934)"
2,10,0_global_0,3_global_0,"LINESTRING (30.362 59.936, 30.364 59.933)"
3,9,0_global_0,4_global_0,"LINESTRING (30.362 59.936, 30.328 59.934)"
4,7,0_global_0,5_global_0,"LINESTRING (30.362 59.936, 30.353 59.932)"
...,...,...,...,...
10002,4,4_building_1045.0,6_building_1045.0,"LINESTRING (30.349 59.919, 30.349 59.919)"
10003,5,4_building_1045.0,7_building_1045.0,"LINESTRING (30.349 59.919, 30.349 59.919)"
10005,4,5_building_1045.0,6_building_1045.0,"LINESTRING (30.349 59.919, 30.349 59.919)"
10006,6,5_building_1045.0,7_building_1045.0,"LINESTRING (30.349 59.919, 30.349 59.919)"
